# Single objective optimization with (sequential) linear programming and `PESTPP-OPT`: bringing in risk evaluation 

In [ ]:
import os
import sys
sys.path.insert(0,"..")
import numpy as np
import matplotlib.pyplot as plt
import pyemu
print(pyemu.__file__)
import flopy
import platform
from pathlib import Path
import shutil
import pandas as pd
from IPython import display


In [ ]:
display.Image("./mv_schematic.png",width=650) 

## Problem setup
 - two city wells in the south of the domain, in combination need to provide 250,000 ft^3/d of water for a city but would like as much as possible
 - the northern well needs to produce 67,000 ft^3/d although it would be acceptable to produce as little as 50,000 ft^3/d - this is for a fancy brewery making nettle-mead syrup and moss beer #soHipster
 - The two stream gages can experience some depletion, but only up to 30%

# Let's build on the previous optimization and make use of the ensemble (stack) from running iES

In [ ]:
pstroot = 'mv_opt_risk.05'

In [ ]:
thisdir = os.getcwd()

# Let's start with our OPT setup - we need to make some changes to it

In [ ]:
template_ws = Path('./simple_opt')
new_ws = Path('./simple_opt_risk')

In [ ]:
if os.path.exists(new_ws):
    shutil.rmtree(new_ws)
shutil.copytree(template_ws, new_ws)

In [ ]:
pst = pyemu.Pst(str(new_ws / 'mv_opt.pst'))

# To consider uncertainty of the objective function outputs, we need to bring in the stack using a couple options

### first let's copy over the ensemble files from the iES run directory

In [ ]:
[shutil.copy2(f'./master_ies_simple/at.3.{cf}.csv',new_ws/ f'at.3.{cf}.stack.csv') for cf in ['obs','par']]


In [ ]:
pst.pestpp_options['opt_recalc_chance_every'] = 100 # let's assume the stack is ok and doesn't need recalculating
pst.pestpp_options['opt_par_stack'] = 'at.3.par.stack.csv'
pst.pestpp_options['opt_obs_stack'] = 'at.3.obs.stack.csv'
pst.pestpp_options['opt_stack_size'] = 100
pst.pestpp_options['opt_risk'] = 0.05


## Also have to free up parameters

In [ ]:
pars = pst.parameter_data
pars.partrans='none'

In [ ]:
obs=pst.observation_data
obs

In [ ]:
pst.control_data.noptmax = 3
pst.write(str(new_ws / f'{pstroot}.pst'), version=2)

In [ ]:
os.chdir(new_ws)
pyemu.os_utils.run(f'pestpp-opt {pstroot}.pst')
os.chdir(thisdir)